# 最大熵模型中文分词

In [89]:
import sys
import re
import logging

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)
logger.info('Hello world!')

INFO:__main__:Hello world!


## 给句子中的每个字符打标签

In [37]:
def tag4_word(word):
    """
    tag4给词语中的每个字符打标签
    """
    tag_word = ''
    if len(word) == 0:
        return ''
    
    if len(word) == 1:
        tag_word = word + '/S'
    elif len(word) == 2:
        tag_word = word[0] + '/B' + word[1] + '/E'
    else:
        tag_word = word[0] + '/B'
        for char in word[1:-1]:
            tag_word = tag_word + char + '/M'
        tag_word = tag_word + word[-1] + '/E'
    return tag_word

def tag4_sentence(sentence):
    """
    tag4给句子中的每个字符打标签
    sentence: 空格分隔个句子。e.g. "我 爱 伟大 的 中国 。"
    """
    words = re.split('\s+', sentence)
    
    tag_word_list = []
    for word in words:
        tag_word = tag4_word(word)
        tag_word_list.append(tag_word)
    return ''.join(tag_word_list)

def detag4_sentence(sentence):
    """
    对tag4打标签的句子进行反解码，输出按空格分隔的句子
    """
    if len(sentence) == 0:
        return ''
    
    sen = sentence.replace('/S', ' ').replace('/E', ' ').replace('/B', '').replace('/M', '')
    return sen

In [93]:
def tag4_data(input_file, output_file):
    """
    对读入文件中的句子打tag4标签，并写出文件
    """
    with open(input_file, 'r') as f:
        train_data = f.readlines()
        train_data_row_num = len(train_data)
        with open(output_file, 'w+') as f:
            f.truncate()
            for row, sentence in enumerate(train_data):
                tag4_sent = tag4_sentence(sentence)
                f.write(tag4_sent + '\n')
                logger.info('tag4标签[%d/%d]. [sentence]=%s, [tag4-sentence]=%s' % (row+1, train_data_row_num, sentence, tag4_sent))
                
tag4_data('../data/pku_training10.txt', '../data/pku_tag4_training10.txt')

INFO:__main__:tag4标签[1/9]. [sentence]=迈向  充满  希望  的  新  世纪  ——  一九九八年  新年  讲话  （  附  图片  １  张  ）  
, [tag4-sentence]=迈/B向/E充/B满/E希/B望/E的/S新/S世/B纪/E—/B—/E一/B九/M九/M八/M年/E新/B年/E讲/B话/E（/S附/S图/B片/E１/S张/S）/S
INFO:__main__:tag4标签[2/9]. [sentence]=中共中央  总书记  、  国家  主席  江  泽民  
, [tag4-sentence]=中/B共/M中/M央/E总/B书/M记/E、/S国/B家/E主/B席/E江/S泽/B民/E
INFO:__main__:tag4标签[3/9]. [sentence]=（  一九九七年  十二月  三十一日  ）  
, [tag4-sentence]=（/S一/B九/M九/M七/M年/E十/B二/M月/E三/B十/M一/M日/E）/S
INFO:__main__:tag4标签[4/9]. [sentence]=１２月  ３１日  ，  中共中央  总书记  、  国家  主席  江  泽民  发表  １９９８年  新年  讲话  《  迈向  充满  希望  的  新  世纪  》  。  （  新华社  记者  兰  红光  摄  ）  
, [tag4-sentence]=１/B２/M月/E３/B１/M日/E，/S中/B共/M中/M央/E总/B书/M记/E、/S国/B家/E主/B席/E江/S泽/B民/E发/B表/E１/B９/M９/M８/M年/E新/B年/E讲/B话/E《/S迈/B向/E充/B满/E希/B望/E的/S新/S世/B纪/E》/S。/S（/S新/B华/M社/E记/B者/E兰/S红/B光/E摄/S）/S
INFO:__main__:tag4标签[5/9]. [sentence]=同胞  们  、  朋友  们  、  女士  们  、  先生  们  ：  
, [tag4-sentence]=同/B胞/E们/S、/S朋/B友/E们/S、/S女/B士/E们/S、/S先/B生/E们/S：/S
INFO:__main__:tag4标签[6/9]. [sentence]=在  １９９８年  来临

## 定义特征函数

In [112]:
def get_char(sentence, i, steps=3):
    """
    获取sentence在位置i处的字符
    """
    words_len = len(sentence) / steps;
    if (i < 0 or i > words_len - 1):
        return '_'
    else:
        return sentence[i*steps]
    
def get_feature1(sentence, i, steps=3):
    """
    特征1：Cn(n=-2,-1,0,1,2)
    """
    words = []
    words.append('C-2=' + get_char(sentence, i - 2, steps))
    words.append('C-1=' + get_char(sentence, i - 1, steps))
    words.append('C0=' + get_char(sentence, i, steps))
    words.append('C1=' + get_char(sentence, i + 1, steps))
    words.append('C2=' + get_char(sentence, i + 2, steps))
    return ''.join(words)

def feature_sentence(sentence, steps=3):
    features = []
    for i in range(len(sentence)//steps):
        # 特征1
        feat = get_feature1(sentence, i, steps)
        # 标签
        label = sentence[i*steps+2]
        if len(feat) > 0:
            features.append((feat, label))
    return features

feature_sentence('是/S中/B国/E发/B展/E历/B史/E上/S非/B常/E重/B要/E的/S', 3)

[('C-2=_C-1=_C0=是C1=中C2=国', 'S'),
 ('C-2=_C-1=是C0=中C1=国C2=发', 'B'),
 ('C-2=是C-1=中C0=国C1=发C2=展', 'E'),
 ('C-2=中C-1=国C0=发C1=展C2=历', 'B'),
 ('C-2=国C-1=发C0=展C1=历C2=史', 'E'),
 ('C-2=发C-1=展C0=历C1=史C2=上', 'B'),
 ('C-2=展C-1=历C0=史C1=上C2=非', 'E'),
 ('C-2=历C-1=史C0=上C1=非C2=常', 'S'),
 ('C-2=史C-1=上C0=非C1=常C2=重', 'B'),
 ('C-2=上C-1=非C0=常C1=重C2=要', 'E'),
 ('C-2=非C-1=常C0=重C1=要C2=的', 'B'),
 ('C-2=常C-1=重C0=要C1=的C2=_', 'E'),
 ('C-2=重C-1=要C0=的C1=_C2=_', 'S')]